## Project Description

In this notebook, I will tell you about building my submission to the [Pump it Up: Data Mining the Water Table](https://www.drivendata.org/competitions/7/pump-it-up-data-mining-the-water-table/) competition hosted by [DrivenData](https://www.drivendata.org/). The competition problem uses data about waterpoints (pumps, wells, etc.) in Tanzania collected by a partnership between [Taarifa](http://taarifa.org/) and the [United Republic of Tanzania Ministry of Water](https://www.maji.go.tz/) to predict if they are currently in need of repair. By better understanding what factors predict issues with water points, we hope to improve access to potable water in Tanzania and reduce operational costs associated with maintaining water infrastructure. 

For the purposes of the competition, the model is assessed in terms of the classification rate, also commonly called [accuracy](https://en.wikipedia.org/wiki/Accuracy_and_precision).The classification rate is a number between zero and one with zero indicating that all prediction were wrong, one indicating all predictions were correct, and 0.7 indicating that, on average, seven out of ten predictions are correct.

All relevant data and code is available in the project [github repository](https://github.com/sethchart/Pump-it-Up-Data-Mining-the-Water-Table).

### Importing Required Modules
Below I collect the tools that I will use throughout the notebook.

In [22]:
from data_utilities import DataWrapper
#Basics
import pandas as pd
import numpy as np
from itertools import combinations
from tqdm.notebook import tqdm

#Plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('notebook')

#Train Test Split
from sklearn.model_selection import train_test_split

# Imputer
from sklearn.impute import SimpleImputer

# Preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PowerTransformer,FunctionTransformer

#Resampler
from imblearn.over_sampling import SMOTE

# Classifiers
from xgboost import XGBClassifier
from sklearn.multiclass import OneVsRestClassifier

#Pipeline
from imblearn.pipeline import Pipeline

#Grid Search
from sklearn.model_selection import GridSearchCV

# Model evaluation
from sklearn.metrics import plot_confusion_matrix

### Importing Data

In [2]:
data = DataWrapper()
X_train = data.X_train
X_test = data.X_test
y_train = data.y_train
y_test = data.y_test

## Conclusions From Data Inspection
After inspecting the data, there are some issues that need to be addressed before we proceed to modeling. 
* There are several variables the explicitly depend on other variables, these should be dropped from the model to avoid issues. There are two reasonable strategies for dealing with these:
    * Keep the variable with the most granular data. There is no information loss associated with this approach, but it will increase model complexity.
    * Keep the variable with the coarser data, thereby aggregating the granular information in the finer variable. This approach reduces model complexity, but it also results in some loss of information.
* There are some variables with a huge number of values. Including these variables will greatly increase the complexity of our model and the time required to train the model.
* The Data Collection and Unknown variable groups should be excluded because any relevant information that they carry is from some unknown lurking variables.
* We will try to deal with all missing values, data encoding, and distributional issues with standard data cleaning procedures.

## Select Columns to Drop from the Model
In this section I will identify columns that should be dropped from the model. Below I have initialized the `drop_cols` list with the columns that will be dropped regardless of my approach to the other issues identified in the conclusion of the previous section.

In [3]:
drop_cols = [
    'date_recorded', 
    'recorded_by',
    'num_private',
    'public_meeting'
]

## Classifying Variables
We will need to pre-process our data in preparation for classification. Pre-processing is different for categorical and numerical variables. In order to implement different pre-pricessing flows, we must first classify all of our variables as categorical or numerical. The function below separates columns into these two classes and excludes any variables that will be dropped from the model.

In [4]:
def classify_columns(drop_cols, df=X_train):
    """Takes a dataframe and a list of columns to drop and returns:
        - cat_cols: A list of categorical columns.
        - num_cols: A list of numerical columns.
    """
    cols = df.columns
    keep_cols = [col for col in cols if col not in drop_cols]
    cat_cols = []
    num_cols = []
    for col in keep_cols:
        if df[col].dtype == object:
            cat_cols.append(col)
        else:
            num_cols.append(col)
    return cat_cols, num_cols

## Dropping Categorical Variables With Many Values

In [15]:
def drop_complex_categorical_variables(drop_cols, df=X_train, threshold= 20):
    cat_cols, num_cols = classify_columns(drop_cols)
    for col in cat_cols:
        if len(df[col].unique()) > threshold:
            drop_cols.append(col)
    return drop_cols

In [16]:
drop_complex_categorical_variables(drop_cols)

['date_recorded',
 'recorded_by',
 'num_private',
 'public_meeting',
 'region',
 'funder',
 'installer',
 'wpt_name',
 'subvillage',
 'lga',
 'ward',
 'scheme_name']

## Dropping Dependent Variables 

The function below will check for pairwise functional dependence between variables and drop variables according to the selected strategy. 

In [24]:
def drop_dependent_categorical_varables(drop_cols, df=X_train, priority='information'):
    cat_cols, num_cols = classify_columns(drop_cols)
    N = len(cat_cols)
    cat_combs = combinations(cat_cols, 2)
    for comb in tqdm(cat_combs):
        dict1 = {}
        dict2 = {}
        col1 = comb[0]
        col2 = comb[1]
        col_dict = {}
        for pair in set(zip(df[col1], df[col2])):
            dict1[pair[0]] = dict1.get(pair[0],0)+1
            dict2[pair[1]] = dict2.get(pair[1],0)+1
        if max(dict1.values()) == 1 and max(dict2.values()) == 1:
            col_dict['fine'] = col1
            col_dict['coarse'] = col2
        elif max(dict1.values()) == 1:
            col_dict['fine'] = col1
            col_dict['coarse'] = col2
        elif max(dict2.values()) == 1:
            col_dict['fine'] = col2
            col_dict['coarse'] = col1
        else:
            continue
        if priority == 'information':
            drop_cols.append(col_dict['coarse'])
        elif priority == 'complexity':
            drop_cols.append(col_dict['fine'])
        else:
            pass
    return drop_cols    

In [25]:
drop_dependent_categorical_varables(drop_cols, priority='information')

['date_recorded',
 'recorded_by',
 'num_private',
 'public_meeting',
 'region',
 'funder',
 'installer',
 'wpt_name',
 'subvillage',
 'lga',
 'ward',
 'scheme_name',
 'extraction_type_group',
 'extraction_type_class',
 'extraction_type_class',
 'management_group',
 'payment_type',
 'quality_group',
 'quantity_group',
 'source_type',
 'source_class',
 'source_class',
 'waterpoint_type_group']

## Modeling Approach

Initial experiments with unoptimized models showed that the [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/index.html) performed best on this data. With that in mind, I want to build an [sklearn pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) to implement data preprocessing and classification. To optimize the model I will use an [sklearn grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to select the best hyper-parameters for the model and perform five-fold [cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)).

# Data Validation
We experimented with both manual and automated feature selection, however neither approach improved model performance. Initially, we has issues with mixed data types in both the `public_meeting` and `permit` columns. The function below converts all categorical variables to strings to eliminate thoes errors. 

In [26]:
cat_cols, num_cols = classify_columns(drop_cols)
print(cat_cols)
print(num_cols)

['basin', 'scheme_management', 'permit', 'extraction_type', 'management', 'payment', 'water_quality', 'quantity', 'source', 'waterpoint_type']
['amount_tsh', 'gps_height', 'longitude', 'latitude', 'region_code', 'district_code', 'population', 'construction_year']


# Building Preprocessors
Below we build a preprocessing step for our pipeline which handles all data processing.

## Categorical Pipeline
The pipeline below executes the following three steps for all of our categorical data.
1. Convert all values in categorical columns to strings. This avoids data type errors in the following steps.
2. Fill all missing values with the string `missing`.
3. One-hot encode all categorical variables. Because this data contains categorical variables with many possible values, it is possible to encounter values in testing data that was not present in the training data. For this reason, we need to set `handel_unknown` to `ignore` so that the encoder will simply ignore unknown values in testing data.

In [27]:
def convert_categorical_to_string(data):
    return pd.DataFrame(data).astype(str)

CategoricalTypeConverter = FunctionTransformer(convert_categorical_to_string)

In [28]:
categorical_pipeline = Pipeline(steps=[
    ('typeConverter', CategoricalTypeConverter),
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('standardizer', OneHotEncoder(handle_unknown='ignore',dtype=float))
])

## Numerical Pipeline
The pipeline below executes two steps:
1. Imputes missing values in any numerical column with the median value from that column.
2. Scales each variable to have mean zero and standard deviation one.

In [29]:
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('standardizer', PowerTransformer())
])

## Preprocessors
The column transformer below implements each of the three possible pre-processing behaviors. 
1. Apply the categorical pipeline.
2. Apply the numerical pipeline.
3. Drop the specified columns.
The if-then statement below ensures that the drop processor is only implemented if there are columns to drop. This is needed since passing an empty `drop_col` list throws an error.

In [30]:
def make_preprocessor(drop_cols=None, complexity_threshold=None, priority=None):
    if drop_cols==None:
        drop_cols = []
    if complexity_threshold != None:
        drop_cols = drop_complex_categorical_variables(drop_cols, threshold=complexity_threshold)
    if priority != None:
        drop_cols = drop_dependent_categorical_varables(drop_cols, priority=priority)
    cat_cols, num_cols = classify_columns(drop_cols)    
    if len(drop_cols) > 0:
        preprocessor = ColumnTransformer(
        transformers=[
            ('numericalPreprocessor', numerical_pipeline, num_cols),
            ('categoricalPreprocessor', categorical_pipeline, cat_cols),
            ('dropPreprocessor', 'drop', drop_cols)
        ])
    else:
        preprocessor = ColumnTransformer(
        transformers=[
            ('numericalPreprocessor', numerical_pipeline, num_cols),
            ('categoricalPreprocessor', categorical_pipeline, cat_cols)
        ])
    return preprocessor

In [32]:
drop_cols = [
    'date_recorded', 
    'recorded_by',
    'num_private',
    'public_meeting'
]
preprocessor = make_preprocessor(drop_cols=drop_cols, complexity_threshold=200, priority='information')

# Building Pipeline
Below we build our main pipeline which executes two steps.
1. Apply preprocessing to the raw data.
2. Fit a one vs rest classifier to the processed data using an eXtreme Gradient Boosted forest model.

In [42]:
random_state = 42
pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('resampler', SMOTE(random_state=random_state)),
        ('classifier', OneVsRestClassifier(estimator=XGBClassifier()))
    ]
)

In [43]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('numericalPreprocessor',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardizer',
                                                                   PowerTransformer())]),
                                                  ['amount_tsh', 'gps_height',
                                                   'longitude', 'latitude',
                                                   'region_code',
                                                   'district_code',
                                                   'population',
                                                   'construction_year']),
                                                 ('categoricalPreprocessor',
                                  

In [44]:
pipeline.score(X_test, y_test)

0.6914141414141414

# Building Parameter Grid
Below we define a grid of hyper-parameters for our pipeline that will be tested in a grid search below.

In [ ]:
parameter_grid = [
    {
        'classifier__estimator': [XGBClassifier()]
        
    }]

# Instantiate Grid Search
Below we instantiate a grid search object which will fit our pipeline for every combination of the parameters defined above. Since the competition uses accuracy as it's measure of model quality, we sill evaluate model performance in terms of accuracy. For each parameter combination, the grid search will also execute five-fold cross validation. 

In order to maximize performance, we will fit our grid search on the full provided training data set and select our best hyper-parameters based on the results of cross validation. For the purposes of local model evaluation, we will then refit the best model on our local training data and use our local testing data to produce a confusion matrix.

In [ ]:
grid_search = GridSearchCV(
    estimator=pipeline, 
    param_grid=parameter_grid, 
    scoring='accuracy', 
    cv=5, 
    verbose=2, 
    n_jobs=-1,
    refit=True
)

# Fit Grid Search
Below we fit our grid search on the full training set and select the best model hyper-parameters. This step takes an Extremely long time to run.

In [ ]:
grid_search.fit(X, y)
model = grid_search.best_estimator_

# Display Results of Grid Search
Below we display the results of our grid search. We pay particular attention to `std_test_score` which will become larger if the model is over-fit. 

In [ ]:
pd.DataFrame(grid_search.cv_results_)

# Make Predictions for Validation
Below we import the testing data provided by the competition. To maximize performance we refit our model on the full training data set. Predictions are formatted and saved to CSV for submission.

In [ ]:
X_validate = pd.read_csv('../data/testing_features.csv', index_col='id')
y_validate = model.predict(X_validate)
df_predictions = pd.DataFrame(y_validate, index=X_validate.index, columns=['status_group'])
df_predictions.to_csv('../predictions/final_model.csv')

# Produce Confusion Matrix
Below we fit the model on our local training data and produce a confusion matrix using the local test data. This provides a reasonable indication of how the model performs. Because the model needs to be fit before producing the matrix, this step will take a long time to run.

In [ ]:
model.fit(X_train, y_train)
fig, ax = plt.subplots()
fig.set_figheight(10)
fig.set_figwidth(10)
plot_confusion_matrix(model, X_test, y_test, ax=ax, normalize='true', include_values=True)
fig.savefig('../images/Confusion_Matrix.png', bbox_inches='tight')